<a href="https://colab.research.google.com/github/tmu-nlp/100knock2021/blob/main/wei/chapter08/knock70.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Task Description   
70. 単語ベクトルの和による特徴量


*   学習データについて，すべての事例xi(r.v.)の特徴ベクトルxiを並べた行列Xと，正解ラベルを並べた行列（ベクトル）Yを作成する。ラベルの種類数をL(L=4)で表す.
*  i番目の事例はTi個の（記事見出しの）単語列(wi,1,wi,2,…,wi,Ti)から構成される。即ち、i番目の事例の記事見出しを，その見出しに含まれる単語のベクトルの平均で表現したものがxiである学習データ、検証データ、評価データそれぞれの特徴量行列及びラベルベクトルを作成し、ファイルに保存 

In [ ]:
#　!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip

In [ ]:
# !unzip NewsAggregatorDataset.zip

In [ ]:
# 読込時のエラー回避のためダブルクォーテーションをシングルクォーテーションに置換
!sed -e 's/"/'\''/g' drive/MyDrive/ColabNotebooks/NLPknock100/newsCorpora.csv > drive/MyDrive/ColabNotebooks/NLPknock100/newsCorpora_re.csv

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# データの読込
df = pd.read_csv('drive/MyDrive/ColabNotebooks/NLPknock100/newsCorpora_re.csv', header=None, sep='\t', names=['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])

# データの抽出
df = df.loc[df['PUBLISHER'].isin(['Reuters', 'Huffington Post', 'Businessweek', 'Contactmusic.com', 'Daily Mail']), ['TITLE', 'CATEGORY']]

# データの分割
train, valid_test = train_test_split(df, test_size=0.2, shuffle=True, random_state=123, stratify=df['CATEGORY'])
valid, test = train_test_split(valid_test, test_size=0.5, shuffle=True, random_state=123, stratify=valid_test['CATEGORY'])

# 事例数の確認
print('【学習データ】')
print(train['CATEGORY'].value_counts())
print('【検証データ】')
print(valid['CATEGORY'].value_counts())
print('【評価データ】')
print(test['CATEGORY'].value_counts())

In [ ]:
from gensim.models import KeyedVectors

# 学習済み単語ベクトルを読み込む
model = KeyedVectors.load_word2vec_format('drive/MyDrive/ColabNotebooks/NLPknock100/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
import string
import torch

def transform_w2v(text):
  table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
  words = text.translate(table).split()  # 記号をスペースに置換後、スペースで分割してリスト化
  vec = [model[word] for word in words if word in model]  # 1語ずつベクトル化

  return torch.tensor(sum(vec) / len(vec))  # 平均ベクトルをTensor型に変換して出力

In [ ]:
# 特徴ベクトルの作成
X_train = torch.stack([transform_w2v(text) for text in train['TITLE']])
X_valid = torch.stack([transform_w2v(text) for text in valid['TITLE']])
X_test = torch.stack([transform_w2v(text) for text in test['TITLE']])

print(X_train.size())
print(X_train)

In [ ]:
# ラベルベクトルの作成
category_dict = {'b': 0, 't': 1, 'e':2, 'm':3}
y_train = torch.tensor(train['CATEGORY'].map(lambda x: category_dict[x]).values)
y_valid = torch.tensor(valid['CATEGORY'].map(lambda x: category_dict[x]).values)
y_test = torch.tensor(test['CATEGORY'].map(lambda x: category_dict[x]).values)

print(y_train.size())
print(y_train)

In [ ]:
# 保存
data_dir = 'drive/MyDrive/ColabNotebooks/NLPknock100/chapter08/data/'
torch.save(X_train, data_dir + 'X_train.pt')
torch.save(X_valid, data_dir + 'X_valid.pt')
torch.save(X_test, data_dir + 'X_test.pt')
torch.save(y_train, data_dir + 'y_train.pt')
torch.save(y_valid, data_dir + 'y_valid.pt')
torch.save(y_test, data_dir + 'y_test.pt')

In [ ]:
!ls drive/MyDrive/ColabNotebooks/NLPknock100/chapter08/data